In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import math
import seaborn as sns
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (16, 6)
#plt.rcParams["figure.autolayout"] = True # equivalent to .tight_layout(); breaks some legends
plt.rcParams['figure.constrained_layout.use'] = True # Experimental

sns.set_style('darkgrid')

def save_to(year, wacc, importer, figure_name, ext='png'):
    p = Path(f'../figures/{year}-{wacc}-{importer}_{figure_name}.{ext}')
    p.parent.mkdir(exist_ok=True)
    return p

In [ ]:
fn = "../results/results.csv"
fn = Path(fn)

df = pd.read_csv(fn, sep=";")#, index_col=[0,1,2,3,4,5,6])
df = df.set_index([c for c in df.columns if c != 'value'], verify_integrity=True)

In [ ]:
# Do a consistency check
for name, group in df.groupby(df.index.names[:-2]):
    
    sys_cost = group.loc[name].loc['general','Total system cost']['value']
    sum_cost = group.loc[name].loc['cost','value'].sum()

    if math.isclose(sys_cost, sum_cost, rel_tol=1e-3) is False:
        raise ValueError(f'Consistency check failed for {name}. '
                         f'Total system cost do https://www.reddit.com/r/nsfw/comments/6creim/hourglass/not match all individual cost components. '
                         f'Individual cost component missing?\n'
                         f'System cost != Individual cost (sum): {sys_cost:.1f} != {sum_cost:.1f}')

In [ ]:
name

In [ ]:
# The following plots are per year, wacc and importer, reindex dataframe accordingly
df = df.reorder_levels(['year','wacc','importer','esc','exporter','category','subcategory'])

In [ ]:
# Plot all ESCs per exporter
for name, group in df.groupby(df.index.names[:3]):
    (year, wacc, importer) = name
    
    tmp = group.loc[name].loc[:,:,'general','Cost per MWh delivered']
    tmp.reset_index().pivot(index='exporter', columns='esc', values='value').plot(kind='bar', #figsize=(30,10), 
                                                                                  ylabel='EUR per MWh delivered', 
                                                                                  title=f'Comparison of ESC costs to {importer} (Year: {year}, WACC assumptions: {wacc})')
    
    plt.ylim(0,df.reset_index().query('subcategory == "Cost per MWh delivered"')['value'].max()*1.02)
    plt.legend(loc='upper left',bbox_to_anchor=(1,1))
    
    plt.savefig(save_to(year, wacc, importer, 'exporter-costs-per-MWh'))
    plt.show()

In [ ]:
# Plot per ESC all exporters
for name, group in df.groupby(df.index.names[:3]):
    (year, wacc, importer) = name
    
    tmp = group.loc[name].loc[:,:,'general','Cost per MWh delivered']
    tmp.reset_index().pivot(index='esc', columns='exporter', values='value').plot(kind='bar',
                                                                                  ylabel='EUR per MWh delivered',
                                                                                  title=f'Comparison of ESC costs to {importer} (Year: {year}, WACC assumptions: {wacc})')
    
    plt.ylim(0,df.reset_index().query('subcategory == "Cost per MWh delivered"')['value'].max()*1.02)
    plt.legend(loc='upper left',bbox_to_anchor=(1,1))
    
    plt.savefig(save_to(year, wacc, importer, 'esc-costs-per-MWh'))
    plt.show()

In [ ]:
# Energy surplus factor by ESC for each exporter
for name, group in df.groupby(df.index.names[:3]):
    (year, wacc, importer) = name
    
    tmp = group.loc[name].loc[:,:,'general','Energy surplus factor']
    tmp.reset_index().pivot(index='exporter', columns='esc', values='value').plot(kind='bar',
                                                                                  ylabel='Energy surplus factor',
                                                                                  title=f'Energy surplus factors by exporter/ESC to {importer}  (Year: {year}, WACC assumptions: {wacc})')
    
    plt.ylim(0,df.reset_index().query('subcategory == "Energy surplus factor"')['value'].max()*1.02)
    plt.legend(loc='upper left',bbox_to_anchor=(1,1))
    
    plt.savefig(save_to(year, wacc, importer, 'energy-surplus-factors'))
    plt.show()

In [ ]:
# These columns will be combined in the following per-ESC investigation
# the dict 'key' will be used as the new column name, the dict 'value' as
# regex based on which the to-aggregate columns will be idenfied using .filter(regex=value).
# Aggregation function for cost: sum()
columns_to_combine = {
    'Shipping': r'ship convoy [0-9]+ cargo',
    'PV/Wind' : r'pvplant|wind',
    'HVDC connection'    : r'HVDC',
    'Battery storage' : r'battery\s(inverter|storage)',
    'H2 (g) pipeline' : r'H2 \(g\) pipeline',
}

# Cutoff value for relative costs %
# default of 0.1 , i.e. cost shares below of 0.1% total costs are removed from relative plot (right subplot)
cutoff = 0.1

for name, group in df.groupby(df.index.names[:4]):
    (year, wacc, importer, esc) = name

    group = group.loc[name].loc[:,'cost',:] # Select cost only

    group = group.reset_index().pivot(index='exporter',columns='subcategory', values='value') # Convert to appropriate form for stacked bar plot and aggregation

    # Aggregate columns for e.g. shipping, battery, RES
    for k,v in columns_to_combine.items():
        tmp = group.filter(regex=v)
        
        # If ESC doesn't contain those components
        if tmp.empty:
            continue
            
        group = group.drop(tmp.columns, axis=1)

        group[k] = tmp.sum(axis=1)
        
    # Alphabetical order of columns
    #group = group.reindex(sorted(group.columns), axis=1)
    
    ## Static ordering of columns; aggregate similar columns (excl. 'imp' and 'exp')
    group = group.groupby(lambda x: x.replace(' (imp)','').replace(' (exp)',''), axis=1).sum()

    ## Predefined order
    sorder = ['PV/Wind', 'electrolysis', 'seawater desalination', 'clean water tank storage', 'Battery storage']
    final_sorder = sorder
    ## Add all other columns
    final_sorder.extend([c for c in group.columns if c not in sorder])
    group = group[final_sorder]
    group /= 120e6 # TODO remove hardcoding (demand)
    
    # Empty canvas for two subplots
    fig, (ax1, ax2) = plt.subplots(1, 2)#, figsize=(40,10))
    fig.suptitle(f'{esc} to {importer} (Year: {year}, WACC assumptions: {wacc})')
    
    # First, left plot: Abs. cost from different exporters
    group.plot(kind='bar', stacked=True, ax=ax1,
               title=f'Absolute costs')
    ax1.legend(loc='upper center', bbox_to_anchor=(0.5,-.15), ncol=2)
    ax1.set_ylim(0, df.reset_index().query('subcategory=="Cost per MWh delivered"')['value'].max())
    
    
    # Prepare data for 2nd subplot
    group *= 120e6 #TODO remove hardcoding (demand)
    tmp = group.divide(df.loc[name].loc[:,'general','Total system cost']['value'], axis='rows') * 100
    # Apply cutouff (for less clogged diagramm)
    tmp = tmp.where(lambda x: x > cutoff).dropna(axis=1, how='all')
    
    # Second, right plot: Relative cost per component for different exporters
    tmp.plot(kind='barh', stacked=True, ax=ax2,
             title=f'Relative costs (below {cutoff}% not shown)')
    ax2.legend(loc='upper center', bbox_to_anchor=(0.5,-.15), ncol=2)
    ax2.set_xlim(0,100)
    ax2.set_xlabel("%")
    
    plt.savefig(save_to(year, wacc, importer, f'{esc}_cost-composition'))
    plt.show()

In [ ]:
# These columns will be combined in the following per-ESC investigation
# the dict 'key' will be used as the new column name, the dict 'value' as
# regex based on which the to-aggregate columns will be idenfied using .filter(regex=value).
# Aggregation function for CFs: mean()
columns_to_combine = {
    'shipping cargo': r'ship convoy [0-9]+ cargo',
    'PV/Wind' : r'pvplant|wind',
    'HVDC connection' : r'HVDC',
    'Battery storage' : r'battery\s(inverter|storage)',
    'H2 (g) pipeline' : r'H2 \(g\) pipeline',
    'methane (g) pipeline' : r'methane \(g\) pipeline',
}

for name, group in df.groupby(df.index.names[:4]):
    (year, wacc, importer, esc) = name

    group = group.loc[name].loc[:,'capacity factor',:] # Select CFs only
    
    # Remove CFs of individual loading/unloading links:
    # These are not relevant as they are exogeneously provided to the model
    # via loading/unloading time and the shipping schedule
    group = group.drop(group.filter(regex='ship convoy [0-9]+ (un)?loading', axis=0).index)

    group = group.reset_index().pivot(index='exporter',columns='subcategory', values='value') # Convert to appropriate form for stacked bar plot and aggregation

    # Aggregate columns for e.g. shipping, battery, RES
    for k,v in columns_to_combine.items():
        tmp = group.filter(regex=v)
        
        # If ESC doesn't contain those components
        if tmp.empty:
            continue
            
        group = group.drop(tmp.columns, axis=1)

        group[k] = tmp.mean(axis=1)
        
    # Alphabetical order of columns
    group = group.reindex(sorted(group.columns), axis=1)
    
    # Empty canvas for single subplot
    fig, ax1 = plt.subplots(1, 1)#, figsize=(40,10))
    fig.suptitle(f'{esc} to {importer} (Year: {year}, WACC assumptions: {wacc})')
    
    group.plot(kind='bar', stacked=False, ax=ax1,
               title=f'Utilisation factor')
    ax1.legend(loc='upper center', bbox_to_anchor=(0.5,-.1), frameon=True, ncol=2)
    
    ax1.set_ylim(0,1.05)
    
    plt.savefig(save_to(year, wacc, importer, f'{esc}_utilisation-factors'))
    plt.show()

In [ ]:
# +++++++++++++++
# Plot capacities
# +++++++++++++++

# These columns will be combined in the following per-ESC investigation
# the dict 'key' will be used as the new column name, the dict 'value' as
# regex based on which the to-aggregate columns will be idenfied using .filter(regex=value).
# Aggregation function for CFs: sum()
columns_to_combine = {
    'shipping cargo': r'ship convoy [0-9]+ cargo',
    'shipping energy & losses': r'transport ship convoy [0-9]+ trip demand & losses',
    'PV/Wind' : r'pvplant|wind',
    #'HVDC connection' : r'HVDC',
    #'Battery storage' : r'battery\s(inverter|storage)',
    #'H2 (g) pipeline' : r'H2 \(g\) pipeline',
    #'methane (g) pipeline' : r'methane \(g\) pipeline',
}

brk = False

for name, group in df.groupby(df.index.names[:4]):
    (year, wacc, importer, esc) = name

    group = group.loc[name].loc[:,'installed capacity',:] # Select capacities only
    

    # Remove CFs of individual loading/unloading links:
    # These are not relevant as they are exogeneously provided to the model
    # via loading/unloading time and the shipping schedule
    group = group.drop(group.filter(regex='ship convoy [0-9]+ (un)?loading', axis=0).index)

    group = group.reset_index().pivot(index='exporter',columns='subcategory', values='value') # Convert to appropriate form for stacked bar plot and aggregation
    
    # Aggregate columns for e.g. shipping, battery, RES
    for k,v in columns_to_combine.items():
        tmp = group.filter(regex=v)
        
        # If ESC doesn't contain those components
        if tmp.empty:
            continue
            
        group = group.drop(tmp.columns, axis=1)

        group[k] = tmp.sum(axis=1)


        
    # Alphabetical order of columns
    group = group.reindex(sorted(group.columns), axis=1)
    
    # Empty canvas for single subplot
    plt.figure(figsize=(30,10))
    ax1 = plt.gca()
    plt.title(f'{esc} to {importer} (Year: {year}, WACC assumptions: {wacc})')
    
    group.plot(kind='bar', stacked=False, ax=ax1,
               title=f'{esc}: Installed capacity',
               log=True)
    
    for p in ax1.patches:
        
        # log plot breaks if patch height (y value) << 1 (leading to large negative values outside the plot)
        # move the label here to the 10^0 log baseline to fix the problem
        y = p.get_height()
        y = 1 if y < 1 else y
        
        ax1.annotate(format(p.get_height(), '.1f'), 
                   (p.get_x() + p.get_width() / 2., y), 
                   ha = 'center', va = 'center', 
                   xytext = (0, 9), 
                   textcoords = 'offset points',
                   rotation=60)
        
    ax1.legend(loc='upper center', bbox_to_anchor=(.5,-.15), frameon=True, ncol=2)
    
    plt.savefig(save_to(year, wacc, importer, f'{esc}_capacities'))
    plt.show()

In [ ]:
## Energy surplus factor by ESC

In [ ]:
df.reset_index().query('subcategory == "Energy surplus factor"')[['exporter','esc','value']].plot(x='exporter',y='value', style=('.'))

In [ ]:
df.reset_index().query('subcategory == "Energy surplus factor"')[['exporter','esc','value']]

In [ ]:
df = df.reorder_levels(['year','wacc','importer','category','subcategory','exporter','esc'])

In [ ]:
import matplotlib

esc_markers = {
    'hvdc':'X',
    'pipeline-h2':'s', 
    'pipeline-ch4':'p', 
    'shipping-lh2':'^', 
    'shipping-lohc':'d', 
    'shipping-lch4':'v', 
    'shipping-lnh3':'<', 
    'shipping-meoh':'>',
}

cmap = matplotlib.cm.get_cmap('tab10')
exp_colors = {
    'AU':cmap(0),
    'AR':cmap(1), 
    'ES':cmap(2), 
    'EG':cmap(3), 
    'MA':cmap(4), 
    'SA':cmap(5), 
    'DK':cmap(6), 
    'DE':cmap(7),
}

In [ ]:
plt.figure(figsize=(10,10))
ax = plt.gca()

tdf = df.loc[2030,'homogeneous','DE']

for name, group in tdf.groupby(['exporter','esc']):
    x = group.loc['general','Cost per MWh delivered'].loc[name]['value']
    y = group.loc['general','Energy surplus factor'].loc[name]['value']

    ax.scatter(x,y,
               marker=esc_markers[name[1]],
               color=exp_colors[name[0]],
               s=50)
    
ax.set_ylabel('Energy surplus factor')
ax.set_xlabel('EUR per MWh delivered')

# Construct legend
legend_esc = [matplotlib.lines.Line2D([], [], color='None', linestyle='None', markersize=0, label='ESC', marker=None)]
legend_esc += [matplotlib.lines.Line2D([], [], color='white', linestyle='None', markersize=10, markeredgecolor='black',
                                      label=n, marker=m) for n,m in esc_markers.items()]

legend_exp = [matplotlib.lines.Line2D([], [], color='None', linestyle='None', markersize=0, label='Exporter', marker=None)]
legend_exp += [matplotlib.lines.Line2D([], [], linestyle='None',markersize=10, marker='h',
                                      label=n, color=c) for n,c in exp_colors.items()]

ax.legend(handles=legend_esc+legend_exp, ncol=2, loc='lower right')

In [ ]:
plt.figure(figsize=(10,10))
ax = plt.gca()

tdf = df.loc[2030,'homogeneous','DE']

for name, group in tdf.groupby(['exporter','esc']):
    x = group.loc['RES', 'Curtailed electricity'].loc[name]['value']/group.loc['RES', 'Total produced electricity'].loc[name]['value']*100.
    y = group.loc['general', 'Energy surplus factor'].loc[name]['value']

    ax.scatter(x,y,
               marker=esc_markers[name[1]],
               color=exp_colors[name[0]],
               s=50)
    
ax.set_ylabel('Energy surplus factor')
ax.set_xlabel('Curtailed electricity [%]')

# Construct legend
legend_esc = [matplotlib.lines.Line2D([], [], color='None', linestyle='None', markersize=0, label='ESC', marker=None)]
legend_esc += [matplotlib.lines.Line2D([], [], color='white', linestyle='None', markersize=10, markeredgecolor='black',
                                      label=n, marker=m) for n,m in esc_markers.items()]

legend_exp = [matplotlib.lines.Line2D([], [], color='None', linestyle='None', markersize=0, label='Exporter', marker=None)]
legend_exp += [matplotlib.lines.Line2D([], [], linestyle='None',markersize=10, marker='h',
                                      label=n, color=c) for n,c in exp_colors.items()]

ax.legend(handles=legend_esc+legend_exp, ncol=2, loc='lower right')

In [ ]:
tdf = df.loc[2030,'homogeneous','DE']
tdf = tdf.reset_index().drop(columns='category')
tdf = tdf.query('subcategory.str.startswith("Installed capacity")', engine='python')
tdf = tdf.pivot(index=['exporter','esc'], columns='subcategory')['value']
display(tdf)

tdf.plot(kind='bar', stacked=True)

In [ ]:
tdf.query('category=="RES"')